In [ ]:
from scipy.io import arff
import pandas as pd

# Chargement dufichier ARFF
data = arff.loadarff('phpMawTba.arff')

# Conversion des données en DataFrame pandas
adult_census = pd.DataFrame(data[0])

# Affichage du DataFrame
print(adult_census.head())

    age     workclass    fnlwgt        education  education-num  \
0  25.0    b'Private'  226802.0          b'11th'            7.0   
1  38.0    b'Private'   89814.0       b'HS-grad'            9.0   
2  28.0  b'Local-gov'  336951.0    b'Assoc-acdm'           12.0   
3  44.0    b'Private'  160323.0  b'Some-college'           10.0   
4  18.0          b'?'  103497.0  b'Some-college'           10.0   

          marital-status            occupation  relationship      race  \
0       b'Never-married'  b'Machine-op-inspct'  b'Own-child'  b'Black'   
1  b'Married-civ-spouse'    b'Farming-fishing'    b'Husband'  b'White'   
2  b'Married-civ-spouse'    b'Protective-serv'    b'Husband'  b'White'   
3  b'Married-civ-spouse'  b'Machine-op-inspct'    b'Husband'  b'Black'   
4       b'Never-married'                  b'?'  b'Own-child'  b'White'   

         sex  capital-gain  capital-loss  hours-per-week    native-country  \
0    b'Male'           0.0           0.0            40.0  b'United-States'

In [ ]:
target_name = "class"
target = adult_census[target_name]

# méthode replace pour convertir les étiquettes
target.replace({b'<=50K': 0, b'>50K': 1}, inplace=True)
print(target.head())

data = adult_census.drop(columns=[target_name])

0    0
1    0
2    1
3    1
4    0
Name: class, dtype: int64


In [ ]:
#Séparation des variables catégorielles et des variables numériques :
from sklearn.compose import make_column_selector as selector

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(data)
categorical_columns = categorical_columns_selector(data)

Nous définissons d'abord les colonnes en fonction de leur type de données :

Le codage à chaud est appliqué aux colonnes catégorielles. De plus, nous utilisons handle_unknown="ignore" pour résoudre les problèmes potentiels dus aux catégories rares.
caractéristiques numériques de mise à l'échelle numérique qui seront standardisées.
Maintenant, nous créons notre ColumnTransfomer en spécifiant trois valeurs : le nom du préprocesseur, le transformateur et les colonnes. Tout d’abord, créons les préprocesseurs pour les parties numériques et catégorielles.

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()

Maintenant, nous créons le transformateur et associons chacun de ces préprocesseurs à leurs colonnes respectives.

In [ ]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    [
        ("one-hot-encoder", categorical_preprocessor, categorical_columns),
        ("standard_scaler", numerical_preprocessor, numerical_columns),
    ]
)

In [ ]:
#Column Transformer peut être combiné avec un classificateur dans une Pipeline :
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

model = make_pipeline(preprocessor, LogisticRegression(max_iter=500))
model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country']),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week'])])),
                ('logisticregression', LogisticRegression(max_iter=500))])

In [ ]:
#Séparation des données
from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(
    data, target, random_state=42
)

In [ ]:
#Train the model:
_ = model.fit(data_train, target_train)
#
data_test.head()
#Prédiction sur les 5 premiers échantillons de l'ensemble test
model.predict(data_test)[:5]
target_test[:5]
model.score(data_test, target_test)

0.8586520350503645

Evaluation du modèle avec Cross Validation

In [ ]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(model, data, target, cv=5)
cv_results
scores = cv_results["test_score"]
print(
    "The mean cross-validation accuracy is: "
    f"{scores.mean():.3f} ± {scores.std():.3f}"
)

The mean cross-validation accuracy is: 0.852 ± 0.003


Installer un modèle plus puissant
Les modèles linéaires sont intéressants car ils sont généralement peu coûteux à former, petits à déployer, rapides à prédire et donnent une bonne base de référence.

Cependant, il est souvent utile de vérifier si des modèles plus complexes, tels qu’un ensemble d’arbres de décision, peuvent conduire à de meilleures performances prédictives. Dans cette section, nous utilisons un tel modèle appelé **arbres d'amplification de gradient** et évaluons ses performances de généralisation. Plus précisément, le modèle scikit-learn que nous utilisons s'appelle **HistGradientBoostingClassifier**.

Pour les modèles arborescents, la gestion des variables numériques et catégorielles est plus simple que pour les modèles linéaires :

nous n'avons pas besoin de mettre à l'échelle les caractéristiques numériques
utiliser un codage ordinal pour les variables catégorielles convient même si le codage aboutit à un ordre arbitraire
Par conséquent, pour HistGradientBoostingClassifier, le pipeline de prétraitement est légèrement plus simple que celui que nous avons vu précédemment pour LogisticRegression :

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.preprocessing import OrdinalEncoder

categorical_preprocessor = OrdinalEncoder(
    handle_unknown="use_encoded_value", unknown_value=-1
)

preprocessor = ColumnTransformer(
    [("categorical", categorical_preprocessor, categorical_columns)],
    remainder="passthrough",
)

model = make_pipeline(preprocessor, HistGradientBoostingClassifier())

#Performance de généralisation
#%%time
_ = model.fit(data_train, target_train)
model.score(data_test, target_test)

0.8791253787568586

Nous pouvons observer que nous obtenons des précisions nettement supérieures avec le modèle Gradient Boosting. C'est souvent ce que nous observons chaque fois que l'ensemble de données comporte un grand nombre d'échantillons et un nombre limité de caractéristiques informatives (par exemple moins de 1 000) avec un mélange de variables numériques et catégorielles.

Cela explique pourquoi les machines à gradient boosté sont très populaires parmi les praticiens de la science des données qui travaillent avec des données tabulaires.